In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [3]:
print(os.listdir("/gdrive/My Drive/Colab Notebooks/ANN/BANANA"))

['ripe', 'unripe', 'overripe', 'rotten']


In [4]:
DATADIR = '/gdrive/My Drive/Colab Notebooks/ANN/BANANA'
CATEGORIES =["overripe","ripe","rotten","unripe"]
IMG_SIZE = 100

In [5]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [6]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [7]:
X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE * IMG_SIZE * 3) # Düzleştirme işlemi
y = np.array(y)

In [8]:
# Veriyi bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [9]:
# Etiketleri one-hot kodlama
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [10]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Önceden eğitilmiş EfficientNetB0 modelini yükleme
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Önceden eğitilmiş modelin katmanlarını dondurma
base_model.trainable = False

# EfficientNetB0'ın üzerine sıralı bir model oluşturma
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Optimizasyon Algoritmasını Ayarla
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Veri Artırma
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
x_train = x_train.reshape((-1, IMG_SIZE, IMG_SIZE, 3))
x_val = x_val.reshape((-1, IMG_SIZE, IMG_SIZE, 3))




# Veri artırma jeneratörünü uygun hale getir
datagen.fit(x_train)

# Modeli eğitme
model.fit(datagen.flow(x_train, y_train, batch_size=32), validation_data=(x_val, y_val), epochs=10)


16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/10
304/304 [==============================] - 191s 609ms/step - loss: 0.3804 - accuracy: 0.8552 - val_loss: 0.2576 - val_accuracy: 0.8999
Epoch 2/10
304/304 [==============================] - 181s 597ms/step - loss: 0.2270 - accuracy: 0.9166 - val_loss: 0.3176 - val_accuracy: 0.8869
Epoch 3/10
304/304 [==============================] - 183s 600ms/step - loss: 0.1946 - accuracy: 0.9291 - val_loss: 0.1805 - val_accuracy: 0.9342
Epoch 4/10
304/304 [==============================] - 187s 616ms/step - loss: 0.1801 - accuracy: 0.9337 - val_loss: 0.1399 - val_accuracy: 0.9490
Epoch 5/10
304/304 [==============================] - 183s 603ms/step - loss: 0.1594 - accuracy: 0.9421 - val_loss: 0.1742 - val_accuracy: 0.9398
Epoch 6/10
304/304 [==============================] - 192s 631ms/step - loss: 0.1526 - accuracy: 0.9441 - val_loss: 0.1530 - val_accuracy: 0.9398
Epoch 7/10
304/304 [==============================] - 193s 

In [11]:
# Test seti üzerinde değerlendirme
x_test = x_test.reshape((-1, IMG_SIZE, IMG_SIZE, 3))
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)


85/85 [==============================] - 29s 333ms/step


In [12]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")


In [13]:
# Sonuçları gösterme
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy:  0.9491839762611276
Precision:  0.9492165073416062
Recall:  0.9491839762611276
F1 Score:  0.9491380227218537

Confusion Matrix:
 [[510  25  10   0]
 [ 19 737  29   6]
 [ 17  10 865  10]
 [  0   0  11 447]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.93       545
           1       0.95      0.93      0.94       791
           2       0.95      0.96      0.95       902
           3       0.97      0.98      0.97       458

    accuracy                           0.95      2696
   macro avg       0.95      0.95      0.95      2696
weighted avg       0.95      0.95      0.95      2696
